In [1]:
import pandas as pd
import os
import glob
import numpy as np
import time
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
from langdetect import DetectorFactory, detect
DetectorFactory.seed = 0
import re

def get_path(country, week):
    base = f'../../../data/03-experiment/{country}/'
    agg = base + f'treatment/followers/00-raw/'
    agg_p = base + f'treatment/followers/01-preprocess/'
    return base, agg, agg_p

import sys
sys.path.insert(0, '../../src/utils')

def lang_detect_na(tweet):
    try:
        lang = detect(tweet)
    except:
        lang = 'NA'
    return lang

def has_text(string):
    return bool(re.search(r'[a-zA-Z]', string))


Use this code to update the overall file for prediction, it takes the last aggregated one and just keeps the posts that havent been preprocessed yet. After this, take the output file and predict on google colab, save it back and aggregate using the following code: 

In [2]:
file_before = 'correct_cases_final.parquet.gzip'
file_new = 'correct_followers_02_08_24.parquet'
file_final = 'correct_cases_02_08_24.parquet.gzip'

In [3]:
for country in ['KE', 'SA']:
    base, agg, agg_p = get_path(country, 'april')
    old = pd.read_parquet(f'{agg_p}/{file_before}')
    old = old[['id']].drop_duplicates(['id']).reset_index(drop = True)
    old['dumdum'] = 1
    old= old[['id', 'dumdum']]
    new = pd.read_parquet(f'../../../../../manual_scraper/data/correct_cases/{country}/{file_new}')
    new['TimeStamp'] = pd.to_datetime(new['TimeStamp'])
    new['id'] = new['follower_handle'].astype(str) + new['TimeStamp'].astype(str)
    new = new.merge(old, on = ['id'], how = 'left')
    new = new[new['dumdum'].isnull()]
    new = new.drop_duplicates('id').reset_index(drop = True)
    new['text'] = new['text'].astype(str)
    new['has_text'] = new['text'].apply(has_text)
    new['has_words'] = new['text'].str.split().apply(lambda x: len(x) > 0)
    print('On to Lang Detection, this might take a few hours')
    new['lang2'] = new['text'].apply(lang_detect_na)
    new.to_parquet(f'{agg_p}/{file_final}', 
                   compression = 'gzip')


On to Lang Detection, this might take a few hours
On to Lang Detection, this might take a few hours
